In [1]:
import pandas as pd 
import re

In [2]:
jobs = pd.read_csv('archive/postings.csv')
company_industries = pd.read_csv('archive/companies/company_industries.csv')


In [3]:
jobs.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,1.715990e+12,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,1.715450e+12,NaN,NaN,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,1.715870e+12,NaN,NaN,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,1.715488e+12,NaN,NaN,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,1.716044e+12,NaN,NaN,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY


In [4]:
jobs.shape

(123849, 28)

In [5]:
jobs.drop(jobs[jobs["company_id"].isna()].index, inplace=True)


In [6]:
jobs = jobs.merge(company_industries, on="company_id")
jobs.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,industry
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,Real Estate
1,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,NaN,NaN,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,Restaurants
2,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,NaN,NaN,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,Law Practice
3,91700727,Downtown Raleigh Alliance,Economic Development and Planning Intern,Job summary:The Economic Development & Plannin...,20.0,HOURLY,"Raleigh, NC",1481176.0,9.0,NaN,...,NaN,NaN,NaN,1.713456e+12,NaN,0,INTERNSHIP,USD,BASE_SALARY,Non-profit Organizations
4,103254301,Raw Cereal,Producer,Company DescriptionRaw Cereal is a creative de...,300000.0,YEARLY,United States,81942316.0,7.0,NaN,...,NaN,NaN,NaN,1.712861e+12,NaN,0,CONTRACT,USD,BASE_SALARY,Design Services


In [7]:
jobs.groupby("title").job_id.count().sort_values(ascending=False)



title
Sales Manager                                                    669
Customer Service Representative                                  362
Project Manager                                                  345
Senior Accountant                                                233
Administrative Assistant                                         229
                                                                ... 
HSE Supervisor (Onsite Denton, TX)                                 1
HSE Supervisor I, Site Mgmt                                        1
HSE Systems Specialist                                             1
HSE and Security Coordinator                                       1
🚀Principal Geotechnical Engineer (ESOP)(Bonus)(Fully Remote)🚀      1
Name: job_id, Length: 71832, dtype: int64

In [13]:
from utils import JOB_TITLES_LIST, vocabulary
#vocabulary = []
#for t in JOB_TITLES_LIST:
#    vocabulary.extend(t.split())
#vocabulary = set(vocabulary)
words = '|'.join([w.lower() for w in vocabulary])
pattern = r'\b(?:{})\b'
valid_jobs = jobs[jobs["title"].apply(lambda x: True if re.search(pattern.format(words), x.lower()) else False)]

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import KMeans
import numpy as np

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def cluster_sentences(sentences, n_clusters=5, model_name='bert-base-uncased'):
    # Load pre-trained model tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Get model output
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform mean pooling to get sentence embeddings
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)

    # Convert embeddings to numpy array
    sentence_embeddings = sentence_embeddings.cpu().numpy()

    # Perform clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(sentence_embeddings)

    return cluster_labels

/Users/alexandrospaliouras/.local/share/virtualenvs/speech_audio_semantic_search-EZLyC-4w/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/alexandrospaliouras/.local/share/virtualenvs/speech_audio_semantic_search-EZLyC-4w/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [14]:

# Example usage
sentences = valid_jobs["title"].unique()

n_clusters = 2
cluster_labels = cluster_sentences(sentences, n_clusters)

# Print the cluster labels for each sentence
for sentence, label in zip(sentences, cluster_labels):
    print(f"Sentence: '{sentence}' is in cluster {label}")

/Users/alexandrospaliouras/.local/share/virtualenvs/speech_audio_semantic_search-EZLyC-4w/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
